In [2]:
!pwd

/home/ec2-user/SageMaker/whisper-sagemaker


In [28]:
%cd /home/ec2-user/SageMaker/whisper-sagemaker

/home/ec2-user/SageMaker/whisper-sagemaker


In [1]:
!ls -l

total 476
-rwxrw-r-- 1 ec2-user ec2-user   1453 Jul  5 06:08 build_and_push.sh
-rw-rw-r-- 1 ec2-user ec2-user 207737 Jul  5 07:23 dialogue1.m4a
-rw-rw-r-- 1 ec2-user ec2-user    524 Jul  5 06:08 Dockerfile
drwxrwxr-x 2 ec2-user ec2-user   4096 Jul  5 06:43 model
-rw-rw-r-- 1 ec2-user ec2-user  15208 Jul  5 06:08 sagemaker_fromlocal.ipynb
-rw-rw-r-- 1 ec2-user ec2-user 244846 Jul  5 07:27 sagemaker_whisper.ipynb
drwxrwxr-x 3 ec2-user ec2-user   4096 Jul  5 07:49 src


In [3]:
ls -a

./                  Dockerfile           sagemaker_fromlocal.ipynb
../                 .git/                sagemaker_whisper.ipynb
build_and_push.sh*  .ipynb_checkpoints/  src/
dialogue1.m4a       model/


In [4]:
!chmod u+x build_and_push.sh

In [15]:
!./build_and_push.sh sagemaker-whisper

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  2.938GB
Step 1/9 : FROM nvidia/cuda:11.7.1-runtime-ubuntu20.04
11.7.1-runtime-ubuntu20.04: Pulling from nvidia/cuda

351b9876: Pulling fs layer 
cf24900f: Pulling fs layer 
573e3fe0: Pulling fs layer 
ed544755: Pulling fs layer 
6c058a44: Pulling fs layer 
ff943437: Pulling fs layer 
6bd606c9: Pulling fs layer 
cb837b22: Pulling fs layer 
1b27f99d: Pull complete 519kB/1.519kBBDigest: sha256:4ff41b20a64e267e9bd9466061711b09adf4807f3d4c656d07009788a56f8178
Status: Downloaded newer image for nvidia/cuda:11.7.1-runtime-ubuntu20.04
 ---> cca775f086be
Step 2/9 : RUN apt-get -y update
 ---> Running in bfef9b6e9ca4
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Ge

In [16]:
%cd model

/home/ec2-user/SageMaker/whisper-sagemaker/model


In [7]:
!curl -O "https://openaipublic.azureedge.net/main/whisper/models/345ae4da62f9b3d59415adc60127b97c714f32e89e936602e85993674d08dcb1/medium.pt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1457M  100 1457M    0     0  2525k      0  0:09:50  0:09:50 --:--:-- 2406k  0:00:14  0:14:29 1693k      0  0:11:36  0:01:12  0:10:24 3159k    0  0:10:09  0:03:00  0:07:09 1916k 1457M   42  624M    0     0  2451k      0  0:10:08  0:04:20  0:05:48 1731k08M    0     0  2483k      0  0:10:00  0:05:33  0:04:27 3011k      0  0:09:53  0:09:39  0:00:14 3160k


In [8]:
!tar -czf model.tar.gz medium.pt

In [9]:
##commented out as this seems unnecessary
# import boto3

# role_name = "SageMaker-local"

# iam = boto3.client("iam")
# role = iam.get_role(RoleName=role_name)["Role"]["Arn"]

In [1]:
import sagemaker as sage

In [2]:
##add this instead of the above
role = sage.get_execution_role()

In [3]:
sess = sage.Session()

In [20]:
model_location = sess.upload_data("./model.tar.gz", key_prefix="whisper/model")

In [21]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-whisper:latest'.format(account, region)

In [22]:
model_name = 'whisper'

container_params = {
    "Image": image,
    "ModelDataUrl": model_location,
}

model = sess.create_model(model_name, role, container_params)

Using already existing model: whisper


In [4]:
sagemaker_client = sess.boto_session.client('sagemaker', region_name='ap-northeast-1')

In [10]:
model = "whisper"
endpoint_config_name = "whisper-config-t2-2xlarge"

create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model, 
            # "InstanceType": "ml.p3.2xlarge",
            "InstanceType": "ml.t2.2xlarge",
            "InitialInstanceCount": 1
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{sess.default_bucket()}/whisper/output"
        },
    }
)

print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

Created EndpointConfig: arn:aws:sagemaker:ap-northeast-1:133132895539:endpoint-config/whisper-config-t2-2xlarge


In [5]:
endpoint_name = 'whisper'
endpoint_config_name = "whisper-config-t2medium"

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)

In [29]:
input_location = sess.upload_data("./dialogue1.m4a", key_prefix="whisper/input")

In [30]:
print(input_location)

s3://sagemaker-ap-northeast-1-133132895539/whisper/input/dialogue1.m4a


In [32]:
import boto3

In [33]:
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='ap-northeast-1')

response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)

print(response)

{'ResponseMetadata': {'RequestId': 'ab6bc893-cbb7-440a-94fe-823f86fc7b35', 'HTTPStatusCode': 202, 'HTTPHeaders': {'x-amzn-requestid': 'ab6bc893-cbb7-440a-94fe-823f86fc7b35', 'x-amzn-sagemaker-outputlocation': 's3://sagemaker-ap-northeast-1-133132895539/whisper/output/d3d9d483-6d41-4200-94ba-f6cc6d3e1c1c.out', 'date': 'Thu, 06 Jul 2023 03:04:37 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'OutputLocation': 's3://sagemaker-ap-northeast-1-133132895539/whisper/output/d3d9d483-6d41-4200-94ba-f6cc6d3e1c1c.out', 'InferenceId': 'aa2adadc-2930-4a78-8304-9c2834967063'}


In [19]:
autoscaling_client = sess.boto_session.client('application-autoscaling') 

variant_name = "variant1"
endpoint_name = 'whisper'

resource_id=f'endpoint/{endpoint_name}/variant/{variant_name}' 

response = autoscaling_client.register_scalable_target(
    ServiceNamespace='sagemaker', 
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=0,  
    MaxCapacity=1
)

response = autoscaling_client.put_scaling_policy(
    PolicyName='Invocations-ScalingPolicy',
    ServiceNamespace='sagemaker', 
    ResourceId=resource_id, 
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 1.0, 
        'CustomizedMetricSpecification': {
            'MetricName': 'ApproximateBacklogSizePerInstance',
            'Namespace': 'AWS/SageMaker',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name }
            ],
            'Statistic': 'Average',
        },
        'ScaleInCooldown': 120,
        'ScaleOutCooldown': 120
    }
)

ClientError: An error occurred (AccessDeniedException) when calling the RegisterScalableTarget operation: User: arn:aws:sts::133132895539:assumed-role/AmazonSageMakerServiceCatalogProductsUseRole/SageMaker is not authorized to perform: application-autoscaling:RegisterScalableTarget on resource: arn:aws:application-autoscaling:ap-northeast-1:133132895539:scalable-target/* because no identity-based policy allows the application-autoscaling:RegisterScalableTarget action

In [5]:
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-west-2')

input_location = f"s3://{sess.default_bucket()}/whisper/input/001-sibutomo.mp3"

response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)